Trabalho Inteligencia Artifical EEL874 - PyTorch

Lucas Manoel de Assis Ventura Trogo
DRE 122168184

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from IPython.display import Image
import cv2
import pytesseract

ModuleNotFoundError: No module named 'cv2'

In [ ]:
#Dados do problema
data = pd.read_csv("/content/Reclamação.csv", encoding="ISO-8859-1", delimiter=";")

In [ ]:
X = data['DESCDEMANDA']
y = data['SERVICO']
# Codificar as etiquetas de serviço
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
# Vetorização do texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Converter para matrizes densas e, em seguida, tensores do PyTorch
X_train = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)

In [ ]:
# Definir o modelo de rede neural
class TextClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
input_dim = X_train.shape[1]
output_dim = len(label_encoder.classes_)
model = TextClassifier(input_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

In [ ]:
y_pred = model(X_test)
y_pred_classes = torch.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(accuracy)

In [ ]:
def predict_service(text):
    text_vector = vectorizer.transform([text])
    text_tensor = torch.tensor(text_vector.toarray(), dtype=torch.float32)
    output = model(text_tensor)
    predicted_class = torch.argmax(output)
    predicted_service = label_encoder.inverse_transform([predicted_class.item()])
    return predicted_service[0]

In [ ]:
def predict_service_from_image(image_path):
    # Carregar a imagem
    image = cv2.imread(image_path)
    # Extrair o texto da imagem usando Tesseract OCR
    extracted_text = pytesseract.image_to_string(image)
    # Fazer a previsão com base no texto extraído
    predicted_service = predict_service(extracted_text)
    return predicted_service, extracted_text

In [ ]:
def extract_text_from_audio(audio_file_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio = r.record(source)
    return r.recognize_google(audio, language="pt-BR")

In [ ]:
def predict_service_from_audio(audio_file_path):
    # Extrair o texto do arquivo de áudio
    extracted_text = extract_text_from_audio(audio_file_path)
    # Fazer a previsão com base no texto extraído
    predicted_service = predict_service(extracted_text)
    return predicted_service

In [ ]:
X = data['DESCDEMANDA']
y = data['SERVICO']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
X_train = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        x = self.fc(x)
        return x


In [ ]:
input_dim = X_train.shape[1]
output_dim = len(label_encoder.classes_)
model = TextClassifier(input_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

In [ ]:
y_pred = model(X_test)
y_pred_classes = torch.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)

In [ ]:
input_text= "quando pago o ipva 2023"
predicted_service = predict_service(input_text)
print(f"Texto de entrada: {input_text}")
print(f"Serviço predito: {predicted_service}")


In [ ]:
predicted_service = predict_service_from_image("/content/imagem.jpeg")
print(f"Serviço predito: {predicted_service}")
# Caminho para a imagem JPEG
caminho_da_imagem = "/content/imagem.jpeg"

In [ ]:
print("Imagem:")
Image(filename=caminho_da_imagem)

In [ ]:
predicted_service = predict_service_from_audio("/content/audio.wav")
print(f"Serviço predito: {predicted_service}")


In [ ]:
text = extract_text_from_audio("/content/audio.wav")
print(text)
